### ОТБОР АВТОМАТИЧЕСКИ СГЕНЕРИРОВАННЫХ ПРИЗНАКОВ

Предположим, вам удалось автоматически сгенерировать большое количество признаков, описывающих набор данных большого временного ряда. Скорее всего, вам не удастся оценить их все при первом же просмотре данных, поэтому всегда не лишне дополнить функцию автоматической генерации признаков возможностью их автоматического отбора. К одним из лучших инструментов отбора признаков относится алгоритм FRESH, который реализован в описанном ранее пакете tsfresh. Этот алгоритм выбирает релевантные признаки на основе масштабируемых критериев проверки гипотез. Необходимость в разработке алгоритма FRESH была вызвана постоянно увеличивающимся объемом данных временных рядов, зачастую подлежащих распределенному хранению с целью последующего распараллеливания вычислений.
Алгоритм оценивает значимость каждого входного признака по отношению к целевой переменной методом вычисления p-значения для каждого из них. После этого p-значения каждого признака оцениваются в совокупности с помощью процедуры Бенджамини-Екутиели, учитывающей такие входные параметры, как уровень допустимых погрешностей, и указывающей, какие из признаков следует сохранить для дальнейшего использования. Процедура Бенджамини-Екутиели — это метод ограничения количества ложных срабатываний, обнаруженных во время проверки гипотез, используемый для получения p-значений на начальном этапе алгоритма FRESH.
Самостоятельная реализация таких процедур — это довольно сложная задача, но воспользовавшись пакетом tsfresh, ее можно решить всего в несколько строк кода. В следующем примере использован шаблон кода, приведенный в документации к модулю. Вначале нужно загрузить данные временных рядов, относящиеся к сбоям в работе робота.


In [1]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 67.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.17 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.17 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
bayesian-optimization 1.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
b

In [2]:
import tsfresh

In [3]:
from tsfresh.examples.robot_execution_failures import download_robot_execution_failures, load_robot_execution_failures
download_robot_execution_failures()
timeseries, y = load_robot_execution_failures()

Признаки извлекаются без дополнительного указания, поэтому пакет автоматически вычисляет их все. В этом смысле он поступает вразрез с предостережениями, приведенными ранее, оставаясь достаточно беспечным относительно экономии вычислительных ресурсов. В тестовом наборе данных не слишком много точек данных, но вы, вероятнее всего, откажетесь от полной их обработки, предварительно не сократив до разумного количества.

In [4]:
from tsfresh import extract_features
extracted_features = extract_features(timeseries, column_id="id", column_sort="time")

Feature Extraction: 100%|██████████| 10/10 [00:32<00:00,  3.25s/it]


Хотя пакет tsfresh позволяет указывать вычисляемые признаки, в настоящем примере мы будем рассматривать все возможные варианты. Заметьте, что всегда можно указать параметры для тех признаков, которые должны приниматься в расчет при вычислении. Все возможности детально описаны и проиллюстрированы в документации.
Проведя полное извлечение для данных — в нашем случае включенных в пакет tsfresh — можно получить огромное количество признаков .


In [5]:
# Пример длинного списка признаков
columns = ['F_x_abs_energy', 'F_x_absolute_sum_of_changes', 
           'F_x_agg_autocorrelation_f_agg_mean', 
           'F_x_agg_autocorrelation_f_agg_median', 
           'F_x_agg_autocorrelation_f_agg_var', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_intercept', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_rvalue', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_slope', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_10_attr_stderr', 
           'F_x_agg_linear_trend_f_agg_max_chunk_len_50_attr_intercept',
           # ... [другие признаки]
           'T_z__variance_larger_than_standard_deviation']

# Получение списка столбцов из extracted_features
columns = extracted_features.columns.tolist()

# Замена списка признаков
extracted_features.columns = columns

In [6]:
print(extracted_features)

    F_x__variance_larger_than_standard_deviation  F_x__has_duplicate_max  \
1                                            0.0                     0.0   
2                                            0.0                     1.0   
3                                            0.0                     0.0   
4                                            0.0                     1.0   
5                                            0.0                     0.0   
..                                           ...                     ...   
84                                           1.0                     1.0   
85                                           1.0                     0.0   
86                                           1.0                     0.0   
87                                           1.0                     1.0   
88                                           1.0                     0.0   

    F_x__has_duplicate_min  F_x__has_duplicate  F_x__sum_values  \
1                   

In [7]:
print(extracted_features.columns)

Index(['F_x__variance_larger_than_standard_deviation',
       'F_x__has_duplicate_max', 'F_x__has_duplicate_min',
       'F_x__has_duplicate', 'F_x__sum_values', 'F_x__abs_energy',
       'F_x__mean_abs_change', 'F_x__mean_change',
       'F_x__mean_second_derivative_central', 'F_x__median',
       ...
       'T_z__fourier_entropy__bins_5', 'T_z__fourier_entropy__bins_10',
       'T_z__fourier_entropy__bins_100',
       'T_z__permutation_entropy__dimension_3__tau_1',
       'T_z__permutation_entropy__dimension_4__tau_1',
       'T_z__permutation_entropy__dimension_5__tau_1',
       'T_z__permutation_entropy__dimension_6__tau_1',
       'T_z__permutation_entropy__dimension_7__tau_1',
       'T_z__query_similarity_count__query_None__threshold_0.0',
       'T_z__mean_n_absolute_max__number_of_maxima_7'],
      dtype='object', length=4698)


Нами получено 4764 столбца значений. Это количество признаков гораздо	больше того, которое можно вычислить вручную, но при работе с реальными наборами данных на их вычисление уйдет заметно больше времени. Решившись на развертывание настолько большого набора признаков, постарайтесь реалистично оценить как вычислительную мощность рабочего оборудования, так собственные способности и скорость анализа полученных результатов. Помните, что выбросы данных во временных рядах могут оказаться особенно сложнымии неудобными для последующего анализа. Обязательно убедитесь, что избранные вами признаки лишены их.